In [1]:
import os
os.chdir( "../")

In [2]:
import psutil
num_cpus = psutil.cpu_count(logical=False)
num_cpus 


8

In [3]:
import pandas as pd
import numpy as np

path = "Dataset/data_train.csv"
df = pd.read_csv(filepath_or_buffer=path,
                               sep=",",
                               header=1,
                               engine='python',
                               names=['UserID', 'ItemID', 'Interaction'])

metric_to_optimize="MAP"

df

,UserID,ItemID,Interaction
0,1,15,1.0
1,1,16,1.0
2,1,133,1.0
3,1,161,1.0
4,1,187,1.0
...,...,...,...
478724,13024,13605,1.0
478725,13024,13823,1.0
478726,13024,15122,1.0
478727,13024,18185,1.0


In [4]:
user_ids = df["UserID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
item_ids = df["ItemID"].unique().tolist()
item2item_encoded = {x: i for i, x in enumerate(item_ids)}
item_encoded2item = {i: x for i, x in enumerate(item_ids)}
df["User"] = df["UserID"].map(user2user_encoded)
df["Item"] = df["ItemID"].map(item2item_encoded)

num_users = len(user2user_encoded)
num_items = len(item_encoded2item)
df["Interaction"] = df["Interaction"].values.astype(np.float32)

# min and max ratings will be used to normalize the ratings later
min_rating = 0.0
max_rating = max(df["Interaction"])

print(
    "Number of users: {}, Number of Items: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_items, min_rating, max_rating
    )
)

Number of users: 12638, Number of Items: 22222, Min rating: 0.0, Max rating: 1.0


In [5]:
userId_unique = df["UserID"].unique()
itemId_unique = df["ItemID"].unique()

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

seeds = [
    7315366, 9428914, 5564236, 
    3420043, 9223036, 851434, 3300082, 7899660, 
    2053768, 4305274, 3709256, 8795594, 1418903, 
    7428566, 157832
]

seeds = seeds[:num_cpus]

urm_all = sps.coo_matrix((df["Interaction"].values, 
                          (df["User"].values, df["Item"].values)))

urm_train_validation = []
urm_test = []
urm_train = []
urm_validation = []

for i in range(len(seeds)):
    a, b = split_train_in_two_percentage_global_sample(urm_all, train_percentage = 0.80,seed=seeds[i])
    c, d = split_train_in_two_percentage_global_sample(a, train_percentage = 0.80,seed=seeds[i])
    urm_train_validation.append(a)
    urm_test.append(b)
    urm_train.append(c)
    urm_validation.append(d)



In [7]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = []
for u in urm_test:
    evaluator_test.append(EvaluatorHoldout(u, cutoff_list=[10], ignore_users=[]))

EvaluatorHoldout: Ignoring 2136 (16.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2128 (16.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2180 (17.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2196 (17.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2202 (17.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2173 (17.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2187 (17.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2162 (17.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [8]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.MatrixFactorization.ImplicitIALSRecommender import ImplicitALSRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender
from Recommenders.Recommender_import_list import *
import optuna

In [9]:
# Replace the following with your MySQL database connection details
host = "db-mysql-nyc3-14396-do-user-15539286-0.c.db.ondigitalocean.com"
port = 25060
database_name = "recsys_optuna"
username = "doadmin"
password = "AVNS_EIQ1D-GbDGJWZPsN3Mt"

# Create an SQLAlchemy engine
mysql_url = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_name}"
mysql_url
storage = optuna.storages.RDBStorage(url=mysql_url)

nmf_name = "NMF8-FOLD"
knn_name = "itemKNN10_DB"
slim_name = "SLIM_EN10_DB"
rp3beta_name = "RP3b10_DB"
p3alpha_name = "P3a10_DB"
hybrid_name = "HYBRID10_DB"
#nmf_study = optuna.load_study(study_name=nmf_name, storage=storage)
#knn_study = optuna.load_study(study_name=knn_name, storage=storage)
#slim_study = optuna.load_study(study_name=slim_name, storage=storage)
#rp3beta_study = optuna.load_study(study_name=rp3beta_name, storage=storage)
#p3alpha_study = optuna.load_study(study_name=p3alpha_name, storage=storage)
#hybrid_study = optuna.load_study(study_name=hybrid_name, storage=storage)

#studies = [nmf_study, knn_study, slim_study, rp3beta_study, p3alpha_study]
#studies = [knn_study, slim_study, rp3beta_study, p3alpha_study]
#recommenders_list = ["Hybrid", "itemKNN", "RP3beta", "P3alpha", "NMF", "SLIM_EN", "TopPop"]
    

In [10]:
metric_to_optimize = "MAP"

In [11]:
#import ProcessPoolExecutor
from concurrent.futures import ProcessPoolExecutor

nmf_study = optuna.create_study(study_name=nmf_name, direction="maximize", storage=storage, load_if_exists=True)

def run_fold(fold, **hyperparams):
    recommender = NMFRecommender(urm_train_validation[fold])
    recommender.fit(**hyperparams)

    metric_result = evaluator_test[fold].evaluateRecommender(recommender)[0][metric_to_optimize].item()
    res = {'Fold': fold, metric_to_optimize: metric_result, **hyperparams}
    print(res)
    return res

def run_fold_wrapper(fold, hyperparams):
    return run_fold(fold, **hyperparams)

sum_map = lambda results: sum(item[metric_to_optimize] for item in results)


def objective(trial):

    params = {
        "num_factors": trial.suggest_int("num_factors",1, 1000),
        "l1_ratio": trial.suggest_float("l1_ratio",low = 0, high = 1),
        "init_type" : trial.suggest_categorical("init_type",["random", "nndsvda"]),
        "beta_loss": trial.suggest_categorical("beta_loss",["frobenius", "kullback-leibler"]),
    }

    with ProcessPoolExecutor(max_workers=num_cpus) as executor:
        # Submit all tasks to the process pool and wait for them to complete
        #fold_results = list(executor.map(lambda fold: run_fold(fold, **current_fit_hyperparameters_dict), range(len(seeds))))
        fold_results = list(executor.map(run_fold_wrapper, range(len(seeds)), [params]*len(seeds)))
    print(fold_results)
    average_score = sum_map(fold_results) / len(fold_results)
    return average_score

[I 2024-01-08 21:18:50,531] A new study created in RDB with name: NMF8-FOLD


In [12]:
nmf_study.enqueue_trial({"num_factors": 254, "l1_ratio": 0.9617319958745, "init_type": "nndsvda", "beta_loss": "frobenius"})
nmf_study.optimize(objective, n_trials=100)

NMFRecommender: URM Detected 226 ( 1.8%) users with no interactions.NMFRecommender: URM Detected 240 ( 1.9%) users with no interactions.NMFRecommender: URM Detected 198 ( 1.6%) users with no interactions.NMFRecommender: URM Detected 189 ( 1.5%) users with no interactions.NMFRecommender: URM Detected 216 ( 1.7%) users with no interactions.

NMFRecommender: URM Detected 216 ( 1.7%) users with no interactions.NMFRecommender: URM Detected 223 ( 1.8%) users with no interactions.NMFRecommender: URM Detected 216 ( 1.7%) users with no interactions.





NMFRecommender: URM Detected 121 ( 0.5%) items with no interactions.NMFRecommender: URM Detected 90 ( 0.4%) items with no interactions.NMFRecommender: URM Detected 108 ( 0.5%) items with no interactions.

NMFRecommender: URM Detected 111 ( 0.5%) items with no interactions.
NMFRecommender: URM Detected 112 ( 0.5%) items with no interactions.NMFRecommender: URM Detected 121 ( 0.5%) items with no interactions.NMFRecommender: URM Detected 118 ( 0.5

[I 2024-01-08 21:22:12,667] Trial 0 finished with value: 0.030079061854780584 and parameters: {'num_factors': 254, 'l1_ratio': 0.9617319958745, 'init_type': 'nndsvda', 'beta_loss': 'frobenius'}. Best is trial 0 with value: 0.030079061854780584.


NMFRecommender: URM Detected 198 ( 1.6%) users with no interactions.NMFRecommender: URM Detected 189 ( 1.5%) users with no interactions.NMFRecommender: URM Detected 226 ( 1.8%) users with no interactions.NMFRecommender: URM Detected 216 ( 1.7%) users with no interactions.NMFRecommender: URM Detected 216 ( 1.7%) users with no interactions.NMFRecommender: URM Detected 240 ( 1.9%) users with no interactions.NMFRecommender: URM Detected 223 ( 1.8%) users with no interactions.NMFRecommender: URM Detected 216 ( 1.7%) users with no interactions.







NMFRecommender: URM Detected 115 ( 0.5%) items with no interactions.NMFRecommender: URM Detected 111 ( 0.5%) items with no interactions.NMFRecommender: URM Detected 121 ( 0.5%) items with no interactions.NMFRecommender: URM Detected 118 ( 0.5%) items with no interactions.NMFRecommender: URM Detected 112 ( 0.5%) items with no interactions.NMFRecommender: URM Detected 121 ( 0.5%) items with no interactions.NMFRecommender: URM Detected 90 ( 0.4%) 

[I 2024-01-08 21:29:41,175] Trial 1 finished with value: 0.029157056206273013 and parameters: {'num_factors': 694, 'l1_ratio': 0.78196543703982, 'init_type': 'nndsvda', 'beta_loss': 'frobenius'}. Best is trial 0 with value: 0.030079061854780584.


NMFRecommender: URM Detected 198 ( 1.6%) users with no interactions.NMFRecommender: URM Detected 223 ( 1.8%) users with no interactions.NMFRecommender: URM Detected 226 ( 1.8%) users with no interactions.NMFRecommender: URM Detected 189 ( 1.5%) users with no interactions.NMFRecommender: URM Detected 240 ( 1.9%) users with no interactions.NMFRecommender: URM Detected 216 ( 1.7%) users with no interactions.NMFRecommender: URM Detected 216 ( 1.7%) users with no interactions.NMFRecommender: URM Detected 216 ( 1.7%) users with no interactions.







NMFRecommender: URM Detected 121 ( 0.5%) items with no interactions.NMFRecommender: URM Detected 108 ( 0.5%) items with no interactions.NMFRecommender: URM Detected 111 ( 0.5%) items with no interactions.NMFRecommender: URM Detected 90 ( 0.4%) items with no interactions.NMFRecommender: URM Detected 118 ( 0.5%) items with no interactions.NMFRecommender: URM Detected 112 ( 0.5%) items with no interactions.NMFRecommender: URM Detected 115 ( 0.5%) 

[I 2024-01-08 21:51:19,626] Trial 2 finished with value: 0.012206104173809958 and parameters: {'num_factors': 827, 'l1_ratio': 0.24089409927786376, 'init_type': 'random', 'beta_loss': 'kullback-leibler'}. Best is trial 0 with value: 0.030079061854780584.


NMFRecommender: URM Detected 226 ( 1.8%) users with no interactions.NMFRecommender: URM Detected 189 ( 1.5%) users with no interactions.NMFRecommender: URM Detected 223 ( 1.8%) users with no interactions.NMFRecommender: URM Detected 216 ( 1.7%) users with no interactions.NMFRecommender: URM Detected 216 ( 1.7%) users with no interactions.


NMFRecommender: URM Detected 198 ( 1.6%) users with no interactions.NMFRecommender: URM Detected 240 ( 1.9%) users with no interactions.NMFRecommender: URM Detected 216 ( 1.7%) users with no interactions.




NMFRecommender: URM Detected 112 ( 0.5%) items with no interactions.NMFRecommender: URM Detected 121 ( 0.5%) items with no interactions.
NMFRecommender: URM Detected 118 ( 0.5%) items with no interactions.NMFRecommender: URM Detected 90 ( 0.4%) items with no interactions.NMFRecommender: URM Detected 121 ( 0.5%) items with no interactions.
NMFRecommender: URM Detected 115 ( 0.5%) items with no interactions.NMFRecommender: URM Detected 108 ( 0.5%

In [23]:
%%time

index = 0

item_knn_recommender = ItemKNNCFRecommender(urm_train_validation[index], verbose=False)
RP3beta_recommender = RP3betaRecommender(urm_train_validation[index], verbose=False)
P3alpha_recommender = P3alphaRecommender(urm_train_validation[index], verbose=False)
SLIM_EN_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(urm_train_validation[index], verbose=False)
NMF_recommender = NMFRecommender(urm_train_validation[index], verbose=False)
TopPop_recommender = TopPop(urm_train_validation[index])
print("ITEMKNN...")
item_knn_recommender.fit(shrink=knn_study.best_params["shrink"], topK=knn_study.best_params["topK"])
print("RP3BETA...")
RP3beta_recommender.fit(alpha=rp3beta_study.best_params["alpha"], beta=rp3beta_study.best_params["beta"], topK=rp3beta_study.best_params["topK"])
print("P3ALPHA...")
P3alpha_recommender.fit(topK=p3alpha_study.best_params["topK"], alpha=p3alpha_study.best_params["alpha"])
print("SLIM...")
SLIM_EN_recommender.fit(workers=12,topK=slim_study.best_params["topK"], l1_ratio=slim_study.best_params["l1_ratio"], alpha=slim_study.best_params["alpha"], verbose=False)
print("NMF...")
NMF_recommender.fit(num_factors = nmf_study.best_params["factors"], l1_ratio = nmf_study.best_params["l1_ratio"], init_type = nmf_study.best_params["init"], beta_loss = nmf_study.best_params["beta_loss"])
print("TopPop...")
TopPop_recommender.fit()

recommenders = {
    "SLIM_EN" : SLIM_EN_recommender,
    "RP3beta" : RP3beta_recommender,
    "itemKNN" : item_knn_recommender,
    "P3alpha" : P3alpha_recommender,
    "NMF" : NMF_recommender,
}
print("HYBRID...")
hybrid_recommender = GeneralizedLinearHybridRecommender(urm_train_validation[index], list(recommenders.values()), verbose=False)
hybrid_recommender.fit([hybrid_study.best_params['weight_{}'.format(i) ] for i, study in enumerate(studies)])

TopPopRecommender: URM Detected 229 ( 1.8%) users with no interactions.
TopPopRecommender: URM Detected 126 ( 0.6%) items with no interactions.
ITEMKNN...
Similarity column 22222 (100.0%), 12576.16 column/sec. Elapsed time 1.77 sec
RP3BETA...
P3ALPHA...
SLIM...
NMF...
TopPop...
HYBRID...
CPU times: user 2min 52s, sys: 4min 7s, total: 6min 59s
Wall time: 1min 13s


In [24]:
evaluator_test[index].evaluateRecommender(hybrid_recommender)[0]["MAP"].item()

EvaluatorHoldout: Processed 10472 (100.0%) in 6.59 sec. Users per second: 1590


0.05167748917748976

In [31]:
%%time

index = 0

item_knn_recommender = ItemKNNCFRecommender(urm_all, verbose=False)
RP3beta_recommender = RP3betaRecommender(urm_all, verbose=False)
P3alpha_recommender = P3alphaRecommender(urm_all, verbose=False)
SLIM_EN_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(urm_all, verbose=False)
NMF_recommender = NMFRecommender(urm_all, verbose=False)
TopPop_recommender = TopPop(urm_all)
print("ITEMKNN...")
item_knn_recommender.fit(shrink=knn_study.best_params["shrink"], topK=knn_study.best_params["topK"])
print("RP3BETA...")
RP3beta_recommender.fit(alpha=rp3beta_study.best_params["alpha"], beta=rp3beta_study.best_params["beta"], topK=rp3beta_study.best_params["topK"])
print("P3ALPHA...")
P3alpha_recommender.fit(topK=p3alpha_study.best_params["topK"], alpha=p3alpha_study.best_params["alpha"])
print("SLIM...")
SLIM_EN_recommender.fit(workers=12,topK=slim_study.best_params["topK"], l1_ratio=slim_study.best_params["l1_ratio"], alpha=slim_study.best_params["alpha"], verbose=False)
print("NMF...")
#NMF_recommender.fit(num_factors = nmf_study.best_params["factors"], l1_ratio = nmf_study.best_params["l1_ratio"], init_type = nmf_study.best_params["init"], beta_loss = nmf_study.best_params["beta_loss"])
print("TopPop...")
TopPop_recommender.fit()

recommenders = {
    "SLIM_EN" : SLIM_EN_recommender,
    "RP3beta" : RP3beta_recommender,
    "itemKNN" : item_knn_recommender,
    "P3alpha" : P3alpha_recommender,
    #"NMF" : NMF_recommender,
}
print("HYBRID...")
hybrid_recommender = GeneralizedLinearHybridRecommender(urm_all, list(recommenders.values()), verbose=False)
hybrid_recommender.fit([hybrid_study.best_params['weight_{}'.format(i) ] for i, study in enumerate(studies)])

ITEMKNN...
Similarity column 22222 (100.0%), 7476.89 column/sec. Elapsed time 2.97 sec
RP3BETA...
P3ALPHA...
SLIM...
NMF...
TopPop...
HYBRID...
CPU times: user 21.4 s, sys: 0 ns, total: 21.4 s
Wall time: 1min 8s


In [32]:
target_users = pd.read_csv("Dataset/data_target_users_test.csv")
target_users.columns = ["UserID"]

In [33]:
item_popularity_encoded = np.ediff1d(urm_all.tocsc().indptr)
item_popularity_encoded = np.sort(item_popularity_encoded)

tar_users = target_users["UserID"].astype(int)
topPop_encoded = item_popularity_encoded[-10:]

submission = []

print(np.unique(df["UserID"].values))

for index, user in enumerate(tar_users):
    if (user not in df["UserID"].values):
        item_list_encoded = topPop_encoded
    else:
        item_list_encoded = hybrid_recommender.recommend(user2user_encoded[user])[:10]
    item_list = []
    for item_encoded in item_list_encoded:
        item_list.append(item_encoded2item[item_encoded])
    submission.append((user, item_list))


def write_submission(submissions):
    with open("./submission_his_bigger_hybrid_NO_NMF.csv", "w") as f:
        f.write("user_id,item_list\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")
            
write_submission(submission)

[    1     2     3 ... 13022 13023 13024]
